In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


In [26]:
df = pd.read_excel('dates_etapes.xlsx')
# Calcul du seuil : 10% de la taille totale des lignes
thresh = len(df) * 0.1

# Suppression des colonnes où au moins 90% des valeurs sont NaN
df = df.dropna(axis=1, thresh=thresh)

# Vérification du résultat
df

,jour,numero_jour,mois,annee,Unnamed: 4,etape,numero_etape,bis,depart,arrivee,Weather
0,Mercredi,25,Juin,1947,:,étape,1,:,Paris,Lille,beau temps
1,Jeudi,26,Juin,1947,:,étape,2,:,Lille,Bruxelles,beau temps
2,Vendredi,27,Juin,1947,:,étape,3,:,Bruxelles,Luxembourg,beau temps
3,Samedi,28,Juin,1947,:,étape,4,:,Luxembourg,Strasbourg,beau temps
4,Dimanche,29,Juin,1947,:,étape,5,:,Strasbourg,Besançon,beau temps
5,Mardi,1er,Juillet,1947,NaN,étape,6,NaN,Besançon,Lyon,beau temps
6,Mercredi,2,Juillet,1947,:,étape,7,:,Lyon,Grenoble,beau temps
7,Jeudi,3,Juillet,1947,:,étape,8,:,Grenoble,Briançon,beau temps
8,Samedi,5,Juillet,1947,:,étape,9,:,Briançon,Digne,beau temps
9,Dimanche,6,Juillet,1947,:,étape,10,:,Digne,Nice,beau temps


In [22]:
def map_weather_category(description):
    description = description.lower()
    if 'vent' in description:
        return 'venteux'
    elif any(chaleur in description for chaleur in ['chaud', 'chaleur', 'forte chaleur']):
        return 'chaud'
    elif 'pluie' in description or 'averses' in description:
        return 'pluvieux'
    elif 'ensoleillé' in description or 'soleil' in description:
        return 'ensoleillé'
    elif 'nuageux' in description or 'couvert' in description:
        return 'nuageux'
    else:
        return 'autre'

In [25]:
# Application de la fonction de mappage
df['Weather_Categorized'] = df['Weather'].apply(map_weather_category)

# Suppression de la colonne 'Weather' originale
df = df.drop(columns=['Weather'])

# Encodage One-Hot
encoder = OneHotEncoder(sparse=False)
weather_encoded = encoder.fit_transform(df[['Weather_Categorized']])
weather_encoded_df = pd.DataFrame(weather_encoded, columns=encoder.get_feature_names_out(['Weather_Categorized']))

# Concaténation avec le DataFrame modifié (sans la colonne 'Weather')
final_df = pd.concat([df, weather_encoded_df], axis=1)
final_df

KeyError: 'Weather'

In [3]:
import requests
from bs4 import BeautifulSoup

url = "http://www.memoire-du-cyclisme.eu/eta_tdf/tdf1982.php"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Recherche des éléments contenant les informations des étapes
stages = soup.find_all('b')  # Les noms des gagnants semblent être en gras

for stage in stages:
    if "km" in stage.text:  # Identifier les lignes contenant les informations des étapes
        winner_info = stage.find_next_siblings('b')[0]  # Le gagnant est le prochain élément en gras
        print(winner_info.text)

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Fonction pour scraper le nom du gagnant de chaque étape
def scrape_winner(year, stage):
    url = f"http://www.memoire-du-cyclisme.eu/eta_tdf/tdf{year}.php"
    response = requests.get(url)
    if response.status_code != 200:
        return "Non disponible"
    soup = BeautifulSoup(response.content, 'html.parser')
    # Recherche des gagnants d'étape
    stages = soup.find_all('b')
    for item in stages:
        if item.get_text().strip().startswith(f"{stage}."):
            winner_info = item.find_next('br').next_sibling
            if winner_info and isinstance(winner_info, str):
                winner_name = winner_info.split(',')[0].strip()
                return winner_name
    return "Non disponible"

# Année du Tour de France
year = '1982'

# Liste des étapes
stages = [str(i) for i in range(1, 23)]  # 22 étapes en 1982

# Scraping du nom du gagnant pour chaque étape
winners = [scrape_winner(year, stage) for stage in stages]

# Affichage des gagnants
for stage, winner in zip(stages, winners):
    print(f"Étape {stage}: {winner}")

Étape 1: Non disponible
Étape 2: Non disponible
Étape 3: Non disponible
Étape 4: Non disponible
Étape 5: Non disponible
Étape 6: Non disponible
Étape 7: Non disponible
Étape 8: Non disponible
Étape 9: Non disponible
Étape 10: Non disponible
Étape 11: Non disponible
Étape 12: Non disponible
Étape 13: Non disponible
Étape 14: Non disponible
Étape 15: Non disponible
Étape 16: Non disponible
Étape 17: Non disponible
Étape 18: Non disponible
Étape 19: Non disponible
Étape 20: Non disponible
Étape 21: Non disponible
Étape 22: Non disponible


In [19]:
from bs4 import BeautifulSoup


url_content = "https://ledicodutour.com/etapes/etapes_par_annees/etapes_1947_1956/etapes_1947.html"

soup = BeautifulSoup(url_content, 'url.parser')

# Trouver les éléments qui contiennent les informations des étapes
winners = []
for tr in soup.find_all('tr'):
    tds = tr.find_all('td')
    if tds and len(tds) > 1:
        stage_info = tds[1].get_text().strip()
        if "Etape :" in stage_info:
            winner_name = stage_info.split(":")[-1].split('(')[0].strip()
            winners.append(winner_name)

# Afficher les gagnants
for winner in winners:
    print(winner)

FeatureNotFound: Couldn't find a tree builder with the features you requested: url.parser. Do you need to install a parser library?

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://ledicodutour.com/etapes/etapes_par_annees/etapes_1947_1956/etapes_1947.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

winners = []
for tr in soup.find_all('tr'):
    cells = tr.find_all('td')
    if len(cells) > 1:
        stage_info = cells[1].get_text().strip()
        if 'Etape :' in stage_info:
            winner = stage_info.split(':')[1].strip()
            winners.append(winner)

for winner in winners:
    print(winner)

Ferdi Kübler
René Vietto
Aldo Ronconi
Jean Robic
Ferdi Kübler
Lucien Teisseire
Jean Robic
Fermo Camellini
René Vietto
Fermo Camellini
Edouard Fachleitner
Henri Massal
Lucien Teisseire
Albert Bourlon
Jean Robic
Giuseppe Tacca
Eloi Tassin
Pietro Tarchini
Raymond Impanis
Maurice Diot
Brik Schotte


In [2]:
def decennie(year):
    decen = []
    borne_inf = 0
    borne_supp = 0
    unite = year % 10
    milliers = year // 1000
    centaines = (year // 100) % 10
    dizaines = (year // 10) % 10
    if unite >= 7:
        borne_inf = milliers*1000 + centaines*100 + dizaines*10 + 7
        borne_supp = milliers*1000 + centaines*100 + (dizaines+1)*10 + 6
    else:
        borne_inf = milliers*1000 + centaines*100 + (dizaines-1)*10 + 7
        borne_supp = milliers*1000 + centaines*100 + dizaines*10 + 6
    return((borne_inf,borne_supp))


print(decennie(1957))

(1957, 1966)


In [4]:



for year in range(1949,2018):
    url = f"https://ledicodutour.com/etapes/classement_etapes/classement_etapes_{decennie(year)[0]}_{decennie(year)[1]}/classement_etapes_{year}.html"
    print(url)


https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1949.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1950.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1951.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1952.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1953.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1954.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1955.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1956.html
https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1957_1966/classement_etapes_1957.html
https://ledicodutour.com/etapes/classement_eta

https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1950.html

In [10]:
import requests
from bs4 import BeautifulSoup

# Remplacer 'url' par l'URL réelle de la page
url = 'https://ledicodutour.com/etapes/classement_etapes/classement_etapes_1947_1956/classement_etapes_1947.html'

# Faire une requête HTTP pour obtenir le contenu de la page
response = requests.get(url)

# Créer un objet BeautifulSoup pour analyser le contenu HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Liste pour stocker les classements des étapes
classements_etapes = []

# Identifier les sections qui contiennent les classements d'étapes
sections_etapes = soup.find_all('p', class_='text-left')

for section in sections_etapes:
    # Rechercher les balises 'strong' dans chaque section
    strong_tags = section.find_all('strong')

    # Variable temporaire pour stocker le nom de l'étape et le classement
    etape_classement = ""

    for tag in strong_tags:
        # Vérifier si le texte contient le nom d'une étape
        if 'km' in tag.text:
            etape_classement = tag.text.strip() + " : "

        # Vérifier si le texte contient le classement d'une étape
        elif any(char.isdigit() for char in tag.text):
            etape_classement += tag.text.strip()
            classements_etapes.append(etape_classement)
            etape_classement = ""

# Afficher les résultats
for classement in classements_etapes:
    print(classement)

21  étapes du 25 juin au 20 juillet 1947
1
20 Saint-Brieuc - Caen 235 km
        21 Caen - Paris 257 km : Le Podium du Tour 1947
Classement des étapes du Tour 1947
1- Paris - Lille 236 km : 1. Ferdi Kübler  en  6h 51' 55"
	  2. Mahé
	  3. Piot à 1'24"
	  4. Vooren à 1'30"
	  5. Klabinski à 1'39"
	  6. Schotte à 1'47"
	  7. Ronconi à 1'52"
	  8. Brambilla à 1'57"
	  9. Robic à 2'09"
	  10. Tacca

	    Classement général :
1- Ferdi Kübler 
	  2- Mahé à 30"
	  3- Piot à 2'24"
2 - Lille - Bruxelles (Bel) 182 km : 1. René Vietto  en 5h 05' 52"
    2. Impanis à 1'41"
    3. Depredomme
    4. Camellii à 8'59"
    5. Ronconi
    6. Breuer
    7. Cogan
    8. Mahé
    9. Brambilla
    10. Huguet à 9'12"

    Classement général : 

    1- René Vietto 
    2- Impanis à 3'28"
    3- Mahé à 6'31"
3- Bruxelles (Bel) - Luxembourg (Lux)
314 km : 1. Aldo Ronconi  en 10h59'13"
2. Cogan à 5'34"
3. Camellini
4. Brambilla
5. Vietto à 6'31"
6. Rossello à 11'08"
7. Latorre à 11'43"
8. Robic à 15'01"
9. Diot 

In [2]:

import pandas as pd
palmares_maillot = pd.read_csv('palmares_maillot.csv')
palmares_maillot

,Grand Prix de la montagne,Par points,Meilleur jeune,Prix de la combativité,Année
0,NaN,NaN,NaN,NaN,1903
1,NaN,NaN,NaN,NaN,1904
2,NaN,NaN,NaN,NaN,1905
3,NaN,NaN,NaN,NaN,1906
4,NaN,NaN,NaN,NaN,1907
...,...,...,...,...,...
105,Romain Bardet,Peter Sagan,Egan Bernal,Julian Alaphilippe,2019
106,Tadej Pogačar,Sam Bennett,Tadej Pogačar,Marc Hirschi,2020
107,Tadej Pogačar,Mark Cavendish,Tadej Pogačar,Franck Bonnamour,2021
108,Jonas Vingegaard,Wout van Aert,Tadej Pogačar,Wout van Aert,2022
